### Preprocessing
Положи рядом с папкой где у тебя все файлы лежат из архива и запусти все ячейки до примеров)
[Ссылка на датасеты](http://files.grouplens.org/datasets/movielens/ml-20m-README.html)

In [1]:
import pandas as pd
import numpy as np

In [2]:
tags = pd.read_csv('ml-20m/tags.csv')
tags['tag'] = tags['tag'].astype(str)
tags['tag'] = tags['tag'].apply(lambda x: x.lower())
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,mark waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [3]:
movies = pd.read_csv('ml-20m/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
id2movie = movies.set_index('movieId').to_dict()['title']

In [5]:
score = pd.read_csv('ml-20m/genome-scores.csv')
score.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [6]:
tags = pd.read_csv('ml-20m/genome-tags.csv')
id2tag = tags.set_index('tagId').to_dict()['tag']

In [22]:
score_test = score[score.relevance >  0.25]

In [7]:
score['tagId'] = score['tagId'].map(id2tag)

In [8]:
score['movieId'] = score['movieId'].map(id2movie)
score.head()

,movieId,tagId,relevance
0,Toy Story (1995),007,0.02500
1,Toy Story (1995),007 (series),0.02500
2,Toy Story (1995),18th century,0.05775
3,Toy Story (1995),1920s,0.09675
4,Toy Story (1995),1930s,0.14675


In [20]:
score.head()

,movieId,tagId,relevance
0,Toy Story (1995),007,0.02500
1,Toy Story (1995),007 (series),0.02500
2,Toy Story (1995),18th century,0.05775
3,Toy Story (1995),1920s,0.09675
4,Toy Story (1995),1930s,0.14675


In [23]:
score_test.to_csv('df_scores.csv')

In [9]:
# score[score.tagId.isin(['007', '1920s'])].groupby('movieId').sum()

### How to search in this dataset

We have score with movieId for which we use id2movie and we have tags, then we use score to define which movies is the best.

As a score we use a $\sum relevance\cdot[tagId == tag]$

I think better to take just sum and this it.

In [10]:
dict_tokens = set(score['tagId'].values)
len(dict_tokens)

1128

In [37]:
import pickle

with open('dict_tags.pkl', 'wb') as fopen:
    pickle.dump(dict_tokens, fopen)

In [11]:
import difflib
difflib.get_close_matches('1920', dict_tokens, cutoff=0.8)

['1920s']

In [12]:
def text_to_tags(text):
    all_tags = set()
    
    text = text.split()
    for word in text:
        match = difflib.get_close_matches(word, dict_tokens, cutoff=0.8)
        all_tags.update(match)
        
    bigramms = [' '.join(x) for x in zip(text[:-1], text[1:])]
    for bigramm in bigramms:
        match = difflib.get_close_matches(bigramm, dict_tokens, cutoff=0.8)
        all_tags.update(match)
    
    return list(all_tags)

In [24]:
def score_func(tags, n):
    return score_test[score_test.tagId.isin(tags)].groupby('movieId').sum().nlargest(n, 'relevance')

In [38]:
check = text_to_tags('romantic and violent')
data = score_func(check, 10)

In [26]:
check = text_to_tags('orphan boy wizard school of magic dark lord')
score_func(check, 10)

,relevance
movieId,
Harry Potter and the Prisoner of Azkaban (2004),4.09050
Harry Potter and the Goblet of Fire (2005),4.07025
Harry Potter and the Deathly Hallows: Part 1 (2010),3.99925
Harry Potter and the Order of the Phoenix (2007),3.89225
Harry Potter and the Chamber of Secrets (2002),3.76875
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),3.65825
Harry Potter and the Half-Blood Prince (2009),3.57800
"Lord of the Rings: The Fellowship of the Ring, The (2001)",3.29400
"Lord of the Rings: The Two Towers, The (2002)",3.28550


In [28]:
movie_titles = pd.read_csv('ml-20m/movies.csv')
movie_titles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [34]:
ids = pd.read_csv('ml-20m/links.csv')
ids['title'] = ids['movieId'].map(id2movie)
ids.head()

,movieId,imdbId,tmdbId,title
0,1,114709,862.0,Toy Story (1995)
1,2,113497,8844.0,Jumanji (1995)
2,3,113228,15602.0,Grumpier Old Men (1995)
3,4,114885,31357.0,Waiting to Exhale (1995)
4,5,113041,11862.0,Father of the Bride Part II (1995)


In [35]:
tmdb = pd.merge(movie_titles, ids, on='title')
tmdb = tmdb[['title', 'imdbId', 'tmdbId']]
tmdb.head()

,title,imdbId,tmdbId
0,Toy Story (1995),114709,862.0
1,Jumanji (1995),113497,8844.0
2,Grumpier Old Men (1995),113228,15602.0
3,Waiting to Exhale (1995),114885,31357.0
4,Father of the Bride Part II (1995),113041,11862.0


In [36]:
tmdb.to_csv('movieID.csv')

### Example of funciton

In [6]:
plot = 'romantic and violent'

In [8]:
import plot_to_movie as ptm

ptm.MoviePlot().plot2movie(plot, 15)

,title,imdbId,tmdbId
551,True Romance (1993),108399,319.0
1760,Heaven's Burning (1997),119272,70581.0
108,Braveheart (1995),112573,197.0
12954,Let the Right One In (Låt den rätte komma in) ...,1139797,13310.0
1068,Bonnie and Clyde (1967),61418,475.0
6932,Revenge (1990),100485,14249.0
1381,"Last of the Mohicans, The (1992)",104691,9361.0
1227,"Femme Nikita, La (Nikita) (1990)",100263,9322.0
6186,In a Lonely Place (1950),42593,17057.0
349,"Crow, The (1994)",109506,9495.0
